# 環境設定

In [ ]:
import os
from google.cloud import bigquery

# SET GCP 金鑰
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=r"/content/tibame-gad253-14-bigquery-c5993cbf5beb.json"

# 初始化BigQuery客戶端
client = bigquery.Client()
print("Client creating using default project: {}".format(client.project))

# 設定資料集
dataset_id = 'tibame_gad253_14_dataset_python'
us_dataset_id = 'tibame_gad253_14_dataset_us'
eu_dataset_id = 'tibame_gad253_14_dataset_eu'

Client creating using default project: tibame-gad253-14-bigquery


In [ ]:
# 連線 ID
# projects/tibame-gad253-14-bigquery/locations/us/connections/tibame_gad253_14_ai

# 服務帳戶 ID
# bqcx-579120420738-xmpo@gcp-sa-bigquery-condel.iam.gserviceaccount.com

# 建立GEMINI模型

In [ ]:
llm_model='llm_model'
connection='projects/tibame-gad253-14-bigquery/locations/us/connections/tibame_gad253_14_ai'
# 定義SQL查詢建立GEMINI模型
query=f"""
CREATE OR REPLACE MODEL `{us_dataset_id}.{llm_model}`
  REMOTE WITH CONNECTION `{connection}`
  OPTIONS (ENDPOINT = 'gemini-2.0-flash-001');

"""
# 印出查詢
print(query)

# 執行查詢
query_job=client.query(query)
query_job.result()


CREATE OR REPLACE MODEL `tibame_gad253_14_dataset_us.llm_model`
  REMOTE WITH CONNECTION `projects/tibame-gad253-14-bigquery/locations/us/connections/tibame_gad253_14_ai`
  OPTIONS (ENDPOINT = 'gemini-2.0-flash-001');




# LLM提示詞-key words

In [ ]:
prompt='Extract the key words from the text below:'
#定義SQL查詢 使用GEMINI
query=f"""
SELECT
*
FROM
  ML.GENERATE_TEXT(
    MODEL `{us_dataset_id}.{llm_model}`,
    (
      SELECT
        CONCAT('{prompt}:', review) AS prompt,
        *
      FROM
        `bigquery-public-data.imdb.reviews`
      LIMIT 5
    ),
    STRUCT(
      0.2 AS temperature,
      100 AS max_output_tokens));

"""
#印出查詢
print(query)

#執行查詢
query_job=client.query(query)
results=query_job.result()

#將results的row解成tuple
rows_data= [tuple(row.values()) for row in results]
#抓出results欄位名稱
columns=[field.name for field in results.schema]
#import panda
import pandas as pd
#顯示資料
df=pd.DataFrame(rows_data,columns=columns)
#取消自動截斷
pd.set_option('display.max_colwidth', None)


SELECT
*
FROM
  ML.GENERATE_TEXT(
    MODEL `tibame_gad253_14_dataset_us.llm_model`,
    (
      SELECT
        CONCAT('Extract the key words from the text below::', review) AS prompt,
        *
      FROM
        `bigquery-public-data.imdb.reviews`
      LIMIT 5
    ),
    STRUCT(
      0.2 AS temperature,
      100 AS max_output_tokens));




In [ ]:
df

,ml_generate_text_result,ml_generate_text_status,prompt,review,split,label,movie_id,reviewer_rating,movie_url,title
0,"{'candidates': [{'avg_logprobs': -0.1585264579922545, 'content': {'parts': [{'text': ""Here's a list of keywords extracted from the text, focusing on the reviewer's opinions and the film's characteristics:\n\n* **Rappers:** (Specifically, negative portrayal)\n* **Gangsters:** (Clichéd portrayal)\n* **Violence:** (Pointless)\n* **Plot:** (Lack of)\n* **Characters:** (Unsympathetic, unlikeable)\n* **Homeless guy:** (Only sympathetic character)\n""}], 'role': 'model'}, 'finish_reason': 'MAX_TOKENS', 'score': -16.16969871520996}], 'create_time': '2025-08-25T08:03:30.569033Z', 'model_version': 'gemini-2.0-flash-001', 'response_id': '0hisaMndIsLwgLUP6r2OgAg', 'usage_metadata': {'billable_prompt_usage': {'text_count': 1034}, 'candidates_token_count': 102, 'candidates_tokens_details': [{'modality': 'TEXT', 'token_count': 102}], 'prompt_token_count': 270, 'prompt_tokens_details': [{'modality': 'TEXT', 'token_count': 270}], 'total_token_count': 372, 'traffic_type': 'ON_DEMAND'}}",,"Extract the key words from the text below::First of all I hate those moronic rappers, who could'nt act if they had a gun pressed against their foreheads. All they do is curse and shoot each other and acting like cliché'e version of gangsters.The movie doesn't take more than five minutes to explain what is going on before we're already at the warehouse There is not a single sympathetic character in this movie, except for the homeless guy, who is also the only one with half a brain.Bill Paxton and William Sadler are both hill billies and Sadlers character is just as much a villain as the gangsters. I did'nt like him right from the start.The movie is filled with pointless violence and Walter Hills specialty: people falling through windows with glass flying everywhere. There is pretty much no plot and it is a big problem when you root for no-one. Everybody dies, except from Paxton and the homeless guy and everybody get what they deserve.The only two black people that can act is the homeless guy and the junkie but they're actors by profession, not annoying ugly brain dead rappers.Stay away from this crap and watch 48 hours 1 and 2 instead. At lest they have characters you care about, a sense of humor and nothing but real actors in the cast.","First of all I hate those moronic rappers, who could'nt act if they had a gun pressed against their foreheads. All they do is curse and shoot each other and acting like cliché'e version of gangsters.The movie doesn't take more than five minutes to explain what is going on before we're already at the warehouse There is not a single sympathetic character in this movie, except for the homeless guy, who is also the only one with half a brain.Bill Paxton and William Sadler are both hill billies and Sadlers character is just as much a villain as the gangsters. I did'nt like him right from the start.The movie is filled with pointless violence and Walter Hills specialty: people falling through windows with glass flying everywhere. There is pretty much no plot and it is a big problem when you root for no-one. Everybody dies, except from Paxton and the homeless guy and everybody get what they deserve.The only two black people that can act is the homeless guy and the junkie but they're actors by profession, not annoying ugly brain dead rappers.Stay away from this crap and watch 48 hours 1 and 2 instead. At lest they have characters you care about, a sense of humor and nothing but real actors in the cast.",test,Negative,tt0105636,1,http://www.imdb.com/title/tt0105636/,Trespass
1,"{'candidates': [{'avg_logprobs': -0.06324665474169182, 'content': {'parts': [{'text': ""Here are the key words extracted from the text:\n\n* Brass pictures/movies\n* brassy\n* visual qualities\n* TV commercials\n* feature films\n* entertain\n* fail\n* erotic\n* Senso '45\n* remake\n* Luchino Visconti\n* senseless love\n* lust\n* Venice\n* Italia

# LLM扁平輸出-只顯示文字

In [ ]:
prompt='Extract the key words from the text below:'
#定義SQL查詢 使用GEMINI&只顯示文字
query=f"""
SELECT
*
FROM
  ML.GENERATE_TEXT(
    MODEL `{us_dataset_id}.{llm_model}`,
    (
      SELECT
        CONCAT('{prompt}:', review) AS prompt,
        *
      FROM
        `bigquery-public-data.imdb.reviews`
      LIMIT 5
    ),
    STRUCT(
      0.2 AS temperature,
      100 AS max_output_tokens,
      TRUE AS flatten_json_output));
"""
#印出查詢
print(query)

#執行查詢
query_job=client.query(query)
results=query_job.result()

#將results的row解成tuple
rows_data= [tuple(row.values()) for row in results]
#抓出results欄位名稱
columns=[field.name for field in results.schema]
#import panda
import pandas as pd
#顯示資料
df=pd.DataFrame(rows_data,columns=columns)
#取消自動截斷
pd.set_option('display.max_colwidth', None)


SELECT
*
FROM
  ML.GENERATE_TEXT(
    MODEL `tibame_gad253_14_dataset_us.llm_model`,
    (
      SELECT
        CONCAT('Extract the key words from the text below::', review) AS prompt,
        *
      FROM
        `bigquery-public-data.imdb.reviews`
      LIMIT 5
    ),
    STRUCT(
      0.2 AS temperature,
      100 AS max_output_tokens,
      TRUE AS flatten_json_output));



In [ ]:
df

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt,review,split,label,movie_id,reviewer_rating,movie_url,title
0,Here are the key words extracted from the text:\n\n* Brass pictures/movies\n* brassy\n* visual qualities\n* TV commercials\n* feature films\n* entertain\n* fail\n* erotic\n* Senso '45\n* remake\n* Luchino Visconti\n* senseless love/lust\n* Venice\n* Italian wars of independence\n* Mussolini\n* German,None,,"Extract the key words from the text below::Brass pictures (movies is not a fitting word for them) really are somewhat brassy. Their alluring visual qualities are reminiscent of expensive high class TV commercials. But unfortunately Brass pictures are feature films with the pretense of wanting to entertain viewers for over two hours! In this they fail miserably, their undeniable, but rather soft and flabby than steamy, erotic qualities non withstanding.Senso '45 is a remake of a film by Luchino Visconti with the same title and Alida Valli and Farley Granger in the lead. The original tells a story of senseless love and lust in and around Venice during the Italian wars of independence. Brass moved the action from the 19th into the 20th century, 1945 to be exact, so there are Mussolini murals, men in black shirts, German uniforms or the tattered garb of the partisans. But it is just window dressing, the historic context is completely negligible.Anna Galiena plays the attractive aristocratic woman who falls for the amoral SS guy who always puts on too much lipstick. She is an attractive, versatile, well trained Italian actress and clearly above the material. Her wide range of facial expressions (signalling boredom, loathing, delight, fear, hate ... and ecstasy) are the best reason to watch this picture and worth two stars. She endures this basically trashy stuff with an astonishing amount of dignity. I wish some really good parts come along for her. She really deserves it.","Brass pictures (movies is not a fitting word for them) really are somewhat brassy. Their alluring visual qualities are reminiscent of expensive high class TV commercials. But unfortunately Brass pictures are feature films with the pretense of wanting to entertain viewers for over two hours! In this they fail miserably, their undeniable, but rather soft and flabby than steamy, erotic qualities non withstanding.Senso '45 is a remake of a film by Luchino Visconti with the same title and Alida Valli and Farley Granger in the lead. The original tells a story of senseless love and lust in and around Venice during the Italian wars of independence. Brass moved the action from the 19th into the 20th century, 1945 to be exact, so there are Mussolini murals, men in black shirts, German uniforms or the tattered garb of the partisans. But it is just window dressing, the historic context is completely negligible.Anna Galiena plays the attractive aristocratic woman who falls for the amoral SS guy who always puts on too much lipstick. She is an attractive, versatile, well trained Italian actress and clearly above the material. Her wide range of facial expressions (signalling boredom, loathing, delight, fear, hate ... and ecstasy) are the best reason to watch this picture and worth two stars. She endures this basically trashy stuff with an astonishing amount of dignity. I wish some really good parts come along for her. She really deserves it.",test,Negative,tt0267261,3,http://www.imdb.com/title/tt0267261/,Black Angel
1,* Costner\n* movie\n* sea rescue\n* characters\n* Ashton Kutcher\n* ghosts\n* obstacle\n* foreshadowing\n,None,,"Extract the key words from the text below::Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The chara

# 輸出關鍵字-繁體中文

In [ ]:
prompt='從以下文字擷取關鍵字'
#定義SQL查詢 使用GEMINI&只顯示文字
query=f"""
SELECT
*
FROM
  ML.GENERATE_TEXT(
    MODEL `{us_dataset_id}.{llm_model}`,
    (
      SELECT
        CONCAT('{prompt}:', review) AS prompt,
        *
      FROM
        `bigquery-public-data.imdb.reviews`
      LIMIT 5
    ),
    STRUCT(
      0.5 AS temperature,
      100 AS max_output_tokens,
      TRUE AS flatten_json_output));
"""
#印出查詢
print(query)

#執行查詢
query_job=client.query(query)
results=query_job.result()

#將results的row解成tuple
rows_data= [tuple(row.values()) for row in results]
#抓出results欄位名稱
columns=[field.name for field in results.schema]
#import panda
import pandas as pd
#顯示資料
df=pd.DataFrame(rows_data,columns=columns)
#取消自動截斷
pd.set_option('display.max_colwidth', None)


SELECT
*
FROM
  ML.GENERATE_TEXT(
    MODEL `tibame_gad253_14_dataset_us.llm_model`,
    (
      SELECT
        CONCAT('從以下文字擷取關鍵字:', review) AS prompt,
        *
      FROM
        `bigquery-public-data.imdb.reviews`
      LIMIT 5
    ),
    STRUCT(
      0.5 AS temperature,
      100 AS max_output_tokens,
      TRUE AS flatten_json_output));



In [ ]:
df

,ml_generate_text_llm_result,ml_generate_text_rai_result,ml_generate_text_status,prompt,review,split,label,movie_id,reviewer_rating,movie_url,title
0,* Brass pictures\n* Erotic qualities\n* Senso '45\n* Luchino Visconti\n* Alida Valli\n* Farley Granger\n* Italian wars of independence\n* Mussolini\n* Anna Galiena\n* SS guy\n* Italian actress\n,None,,"從以下文字擷取關鍵字:Brass pictures (movies is not a fitting word for them) really are somewhat brassy. Their alluring visual qualities are reminiscent of expensive high class TV commercials. But unfortunately Brass pictures are feature films with the pretense of wanting to entertain viewers for over two hours! In this they fail miserably, their undeniable, but rather soft and flabby than steamy, erotic qualities non withstanding.Senso '45 is a remake of a film by Luchino Visconti with the same title and Alida Valli and Farley Granger in the lead. The original tells a story of senseless love and lust in and around Venice during the Italian wars of independence. Brass moved the action from the 19th into the 20th century, 1945 to be exact, so there are Mussolini murals, men in black shirts, German uniforms or the tattered garb of the partisans. But it is just window dressing, the historic context is completely negligible.Anna Galiena plays the attractive aristocratic woman who falls for the amoral SS guy who always puts on too much lipstick. She is an attractive, versatile, well trained Italian actress and clearly above the material. Her wide range of facial expressions (signalling boredom, loathing, delight, fear, hate ... and ecstasy) are the best reason to watch this picture and worth two stars. She endures this basically trashy stuff with an astonishing amount of dignity. I wish some really good parts come along for her. She really deserves it.","Brass pictures (movies is not a fitting word for them) really are somewhat brassy. Their alluring visual qualities are reminiscent of expensive high class TV commercials. But unfortunately Brass pictures are feature films with the pretense of wanting to entertain viewers for over two hours! In this they fail miserably, their undeniable, but rather soft and flabby than steamy, erotic qualities non withstanding.Senso '45 is a remake of a film by Luchino Visconti with the same title and Alida Valli and Farley Granger in the lead. The original tells a story of senseless love and lust in and around Venice during the Italian wars of independence. Brass moved the action from the 19th into the 20th century, 1945 to be exact, so there are Mussolini murals, men in black shirts, German uniforms or the tattered garb of the partisans. But it is just window dressing, the historic context is completely negligible.Anna Galiena plays the attractive aristocratic woman who falls for the amoral SS guy who always puts on too much lipstick. She is an attractive, versatile, well trained Italian actress and clearly above the material. Her wide range of facial expressions (signalling boredom, loathing, delight, fear, hate ... and ecstasy) are the best reason to watch this picture and worth two stars. She endures this basically trashy stuff with an astonishing amount of dignity. I wish some really good parts come along for her. She really deserves it.",test,Negative,tt0267261,3,http://www.imdb.com/title/tt0267261/,Black Angel
1,* Costner\n* movie\n* sea rescue\n* characters\n* Ashton Kutcher\n* ghosts\n* overconfident\n* obstacle\n* foreshadowing\n,None,,"從以下文字擷取關鍵字:Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else

# 建立NLP自然語言模型

In [ ]:
nlpmodel = 'mynlpmodel'
#定義SQL查詢 建立NLP模型
query=f"""
CREATE OR REPLACE MODEL `{us_dataset_id}.{nlpmodel}`
REMOTE WITH CONNECTION `{connection}`
OPTIONS (REMOTE_SERVICE_TYPE = 'CLOUD_AI_NATURAL_LANGUAGE_V1');
"""
#印出查詢
print(query)

#執行查詢
query_job=client.query(query)
query_job.result()


CREATE OR REPLACE MODEL `tibame_gad253_14_dataset_us.mynlpmodel`
REMOTE WITH CONNECTION `projects/tibame-gad253-14-bigquery/locations/us/connections/tibame_gad253_14_ai`
OPTIONS (REMOTE_SERVICE_TYPE = 'CLOUD_AI_NATURAL_LANGUAGE_V1');



# 建立輸入資料表

In [ ]:
table_id = 'mynlptable'
schema = [
    bigquery.SchemaField("comment", "STRING"),
    bigquery.SchemaField("text_contnet", "STRING"),
]

# 定義表格參數
table_ref = client.dataset(us_dataset_id).table(table_id)
table = bigquery.Table(table_ref, schema=schema)

# 建立資料表
table = client.create_table(table)
print(f"table: {table.table_id} ,created")

# 輸入文字進資料表

In [ ]:
text1='20年後、やったことよりも、やらなかったことをもっと後悔するだろう。'
text2='Fais ce que tu veux faire et va là où tu veux aller.'
# 定義資料
rows_to_insert = [
    {"comment": text1, "text_contnet": text1},
    {"comment": text2, "text_contnet": text2}
]

# 插入資料
errors = client.insert_rows_json(f"{us_dataset_id}.{table_id}", rows_to_insert)

if errors == []:
    print("data insert successfully")
else:
    print(f"error: {errors}")

# NLP情感分析

In [ ]:
#定義SQL查詢執行NLP情感分析
query=f"""
SELECT *
FROM ML.UNDERSTAND_TEXT(
  MODEL `{us_dataset_id}.{nlpmodel}`,
  TABLE `{us_dataset_id}.{table_id}`,
  STRUCT('analyze_sentiment' AS nlu_option)
);

"""
#印出查詢
print(query)

#執行查詢
query_job=client.query(query)
results=query_job.result()

#將results的row解成tuple
rows_data= [tuple(row.values()) for row in results]
#抓出results欄位名稱
columns=[field.name for field in results.schema]
#import panda
import pandas as pd
#顯示資料
df=pd.DataFrame(rows_data,columns=columns)
#取消自動截斷
pd.set_option('display.max_colwidth', None)


SELECT *
FROM ML.UNDERSTAND_TEXT(
  MODEL `tibame_gad253_14_dataset_us.mynlpmodel`,
  TABLE `tibame_gad253_14_dataset_us.mynlptable`,
  STRUCT('analyze_sentiment' AS nlu_option)
);




BadRequest: 400 Invalid table-valued function ML.UNDERSTAND_TEXT
ML.UNDERSTAND_TEXT expects the input table to contain a column named text_content of type STRING. at [3:6]; reason: invalidQuery, location: query, message: Invalid table-valued function ML.UNDERSTAND_TEXT
ML.UNDERSTAND_TEXT expects the input table to contain a column named text_content of type STRING. at [3:6]

Location: US
Job ID: a123cf17-6009-4337-af7d-8fc1405044ce


In [ ]:
df

# 語法分析

In [ ]:
#定義SQL查詢執行NLP語法分析
query=f"""
SELECT *
FROM ML.UNDERSTAND_TEXT(
  MODEL `{us_dataset_id}.{nlpmodel}`,
  TABLE `{us_dataset_id}.{table_id}`,
  STRUCT('ANALYZE_SYNTAX' AS nlu_option)
);

"""
#印出查詢
print(query)

#執行查詢
query_job=client.query(query)
results=query_job.result()

#將results的row解成tuple
rows_data= [tuple(row.values()) for row in results]
#抓出results欄位名稱
columns=[field.name for field in results.schema]
#import panda
import pandas as pd
#顯示資料
df=pd.DataFrame(rows_data,columns=columns)
#取消自動截斷
pd.set_option('display.max_colwidth', None)


SELECT *
FROM ML.UNDERSTAND_TEXT(
  MODEL `tibame_gad253_14_dataset_us.mynlpmodel`,
  TABLE `tibame_gad253_14_dataset_us.mynlptable`,
  STRUCT('ANALYZE_SYNTAX' AS nlu_option)
);




BadRequest: 400 Invalid table-valued function ML.UNDERSTAND_TEXT
ML.UNDERSTAND_TEXT expects the input table to contain a column named text_content of type STRING. at [3:6]; reason: invalidQuery, location: query, message: Invalid table-valued function ML.UNDERSTAND_TEXT
ML.UNDERSTAND_TEXT expects the input table to contain a column named text_content of type STRING. at [3:6]

Location: US
Job ID: b19441b8-6cc9-4e42-974b-fc85c59da25d


# NLP分析-子查詢SELECT

In [ ]:
#定義SQL查詢
query=f"""
SELECT *
FROM ML.UNDERSTAND_TEXT(
  MODEL `{us_dataset_id}.{nlpmodel}`,
  (SELECT comment AS text_content FROM `{us_dataset_id}.{table_id}`),
  STRUCT('analyze_syntax' AS nlu_option)
);
"""
#印出查詢
print(query)

#執行查詢
query_job=client.query(query)
results=query_job.result()

#將results的row解成tuple
rows_data= [tuple(row.values()) for row in results]
#抓出results欄位名稱
columns=[field.name for field in results.schema]
#import panda
import pandas as pd
#顯示資料
df=pd.DataFrame(rows_data,columns=columns)
#取消自動截斷
pd.set_option('display.max_colwidth', None)


SELECT * 
FROM ML.UNDERSTAND_TEXT(
  MODEL `tibame_gad253_14_dataset_us.mynlpmodel`,
  (SELECT comment AS text_content FROM `tibame_gad253_14_dataset_us.mynlptable`),
  STRUCT('analyze_syntax' AS nlu_option)
);



In [ ]:
df

# NLP分析-IMDB評論

In [ ]:
#定義SQL查詢
query=f"""
SELECT *
FROM ML.UNDERSTAND_TEXT(
  MODEL `{us_dataset_id}.{nlpmodel}`,
  (SELECT review AS text_content FROM `bigquery-public-data.imdb.reviews` LIMIT 5),
  STRUCT('analyze_syntax' AS nlu_option)
);

"""
#印出查詢
print(query)

#執行查詢
query_job=client.query(query)
results=query_job.result()

#將results的row解成tuple
rows_data= [tuple(row.values()) for row in results]
#抓出results欄位名稱
columns=[field.name for field in results.schema]
#import panda
import pandas as pd
#顯示資料
df=pd.DataFrame(rows_data,columns=columns)
#取消自動截斷
pd.set_option('display.max_colwidth', None)

In [ ]:
df